In [ ]:
import pandas as pd
import numpy as np

In [ ]:
##### 데이터 불러오기 #####

df = pd.read_csv('pill_processed.csv')

PILLS = df['itemName'].values.tolist()

NORMAL = {'인사': ['안녕'],
          '감사': ['고마워', '감사', '고맙']}

KEYWORDS = {0:['회사', '제조사'],
            1:['이름', '제품명'],
            2:['효과', '효능', '약효'],
            3:['용법', '복용', '얼마나', '언제', '적정'],
            4:['부작용'],
            5:['보관'],
            6:['주의', '경고', '유의'],}

In [ ]:
##### 유틸리티 함수 #####

# 약 이름에 따라 알맞는 row index를 반환
def getRIndex(pill):
    return df.index[df['itemName'] == pill][0]

# 제시된 키워드에 따라 알맞는 column index를 반환
def getCIndex(keyword):
    for k, v in KEYWORDS.items():
        if keyword in v:
            return k
    return -1

# 약 이름과 키워드에 따라 알맞는 data를 반환
def getData(pill, keyword):
    ridx = getRIndex(pill)
    cidx = getCIndex(keyword)
    return df.iat[ridx, cidx]

# 주어진 텍스트에서 조사를 제거한 단어들을 반환
def text2words(text):
    words = text.split()
    
    for i in range(len(words)):
        if words[i].strip()[-1] in '은는이가을를의':
            words[i] = words[i][:-1]
    
    return words

In [ ]:
##### 주요 기능 함수 #####

# 제시된 증상에 맞는 약 이름 list를 반환
# Return 약 이름 list for 해당 약 있음
# Return -1 for 해당 약 없음
def whichPill(fac):
    pills = []
    cnt = 0
    for i in range(len(df)):
        efcy = df.iat[i, 2]
        if fac in efcy:
            pills.append(df.iat[i, 1])
            cnt += 1
    
    if cnt == 0:
        return -1
    return pills

def text2cure(text):
    words = text2words(text)
    for word in words:
        whichPill(word)

# 입력 텍스트에 응답 키워드가 들어있는지 확인하여 키워드를 반환
# Return -1 for 키워드 없음
# Return 0 for 키워드 다수
# Return 키워드 for 키워드 하나
def text2key(text):
    cnt = 0
    words = []
    for k, v in KEYWORDS.items():
        for keyword in v:
            if keyword in text:
                cnt += 1
                words.append(keyword)
                break
    
    if cnt == 0:
        return -1
    if cnt == 1:
        return words[0]
    return 0

# 입력 텍스트에 약 이름이 들어있는지 확인하여 약 이름을 반환
# Return -1 for 약 이름 없음
# Return 약 이름 리스트 for 약 이름 다수
# Return 약 이름 for 약 이름 하나
def text2pill(text):
    cnt = 0
    pills = []
    
    words = text2words(text)
    
    for item in words:
        for pill in PILLS:
            if item in pill:
                cnt += 1
                pills.append(pill)
    
    if cnt == 0:
        return -1
    if cnt == 1:
        return pills[0]
    return pills

In [ ]:
##### 초기 메시지 #####

def firstMessage():
    res = """안녕하세요? 약 도우미 챗봇입니다.
약의 보관법, 복용방법, 부작용 등 특정 약에 대해 궁금한 사항부터
특정 증상으로 아플 때 어떠한 약의 도움을 받을 수 있는지,
약에 관해 궁금한 사항은 무엇이든 물어봐 주세요.

예시 1) 약 "냠냠정"의 보관법이 궁금할 때: (약 이름, 궁금한 사항)
    => 냠냠정의 보관법은 뭐야?
    => 냠냠정 보관법
    => 냠냠정 어떻게 보관해야 해? 등
    
예시 2) 배 아플 때 어떤 약을 사야 하는지 궁금할 때: (증상)
    => 복통
    => 복통약 알려줘
    => 복통에 먹는 약 등"""
    
    return res

In [ ]:
##### main격 함수 #####

def respond(text):
    keyword = text2key(text)
    pill = text2pill(text)
    illness = text2
    if (keyword == -1 or pill == -1):
        res = "무슨 말인지 모르겠어요."
    elif (keyword == 0):
        res = "한 번에 하나씩 질문해 주세요."
    elif (type(pill) is list):
        pill_str = '[' + ', '.join(pill) + ']'
        res = "아래 약 목록 중 어느 약 말씀이신가요?\n" + pill_str
    else:
        res = pill + "의 " + keyword + "(은)는: \n" + getData(pill, keyword)
    return res

In [ ]:
# TEST CODE
    
text1 = '안녕하세요, 아네모정의 보관은 어떻게 하나요?'
text2 = '이지엔 보관방법'
text3 = '안녕하세요'
text4 = '아네모정 보관방법 제조사'
text5 = '편두통약이 '

print(firstMessage(), '\n')

print(respond(text1), '\n')
print(respond(text2), '\n')
print(respond(text3), '\n')
print(respond(text4), '\n')
print(whichPill('편두통'))